# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
# Reading in CSV from part one
city_data = pd.read_csv("city_data.csv")
city_data

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,Hermanus,ZA,-34.42,19.23,58.69,95,4.41,75
1,Kaitangata,NZ,-46.28,169.85,53.08,75,7.78,89
2,Fianarantsoa,MG,-21.43,47.08,59.90,70,5.26,99
3,Barrow,US,71.29,-156.79,-5.80,83,12.75,1
4,Española,US,35.99,-106.08,66.96,16,11.41,1
...,...,...,...,...,...,...,...,...
535,Tacuarembó,UY,-31.73,-55.98,64.04,58,8.12,56
536,San Cristobal,VE,7.77,-72.22,91.40,52,13.87,20
537,Wattegama,LK,6.80,81.48,74.10,87,1.74,10
538,Senanga,ZM,-16.12,23.27,67.30,65,6.49,65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Creating locations list from city_data df to use in heatmap
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]

# Creating heatmap layer
fig = gmaps.figure()
humid_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add heatmap to fig
fig.add_layer(humid_layer)

# Display fig
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# Getting temps between 70 and 80 degrees
above_70 = city_data.loc[city_data["Temperature"] > 70]
good_temp = above_70.loc[above_70["Temperature"] < 80]

# Taking good_temp DF and finding all cities with wind speed under 10 mph
low_wind = good_temp.loc[good_temp["Wind Speed"] < 10]

# Taking low_wind DF and finding all cities with 0% cloudiness to complete the good_weather DF
good_weather = low_wind.loc[low_wind["Cloudiness"] == 0]
good_weather

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
44,São Filipe,CV,14.90,-24.50,76.30,66,8.70,0
186,Rafaela,AR,-31.25,-61.49,71.76,32,1.99,0
206,Bataiporã,BR,-22.30,-53.27,73.76,51,3.85,0
383,Balabac,PH,7.99,117.06,79.74,71,6.64,0
457,Najrān,SA,17.49,44.13,75.20,19,4.18,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# Setting up hotel df
hotel_df = good_weather
hotel_df = hotel_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key 
}
  
for index, row in hotel_df.iterrows():
    # Get lat, lng to set up location param
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # adding location key/value pair to params dictionary so we can correctly iterate over cities
    params["location"] = f"{lat},{lng}"
    
    # defining base_url for api call
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # using requests.get to perform api call to get nearest hotels for each city location
    hotel_name = requests.get(base_url, params=params)
    
    # printing list of json objects 
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Try/except statement to handle any missing data fields/results
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field or result... skipping.")    


hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.8995932,
                    "lng": -24.4981834
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.90095017989272,
                        "lng": -24.49681982010728
                    },
                    "southwest": {
                        "lat": 14.89825052010728,
                        "lng": -24.49951947989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "e60d0e51e48b10c1e7ac2dc8aebf17bce36feda4",
            "name": "Hotel Xaguate",
            "photos": [
                {
                    "height": 1080,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101446859031753061453\">Corrie 

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.24805,
                    "lng": -61.490844
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.24671762010728,
                        "lng": -61.48939352010727
                    },
                    "southwest": {
                        "lat": -31.24941727989273,
                        "lng": -61.49209317989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "74b957fef7d4695bf2e0f4bf0f50cf31aadcbfeb",
            "name": "Am\u00e9rian Rafaela Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3438,
                    "html_attributions": [
           

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -22.2886503,
                    "lng": -53.2772953
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.28711822010728,
                        "lng": -53.27600387010727
                    },
                    "southwest": {
                        "lat": -22.28981787989272,
                        "lng": -53.27870352989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "40fa4327476dc932bac8eea6635bfb48c18962d5",
            "name": "Hotel Vale do Ivinhema",
            "photos": [
                {
                    "height": 484,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/10747601034894090325

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Hotel Name
0,São Filipe,CV,14.90,-24.50,76.30,66,8.70,0,Hotel Xaguate
1,Rafaela,AR,-31.25,-61.49,71.76,32,1.99,0,Amérian Rafaela Hotel
2,Bataiporã,BR,-22.30,-53.27,73.76,51,3.85,0,Hotel Vale do Ivinhema
3,Balabac,PH,7.99,117.06,79.74,71,6.64,0,Sing and Swing Lodge
4,Najrān,SA,17.49,44.13,75.20,19,4.18,0,


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[item for item in hotel_info])

# Add hotel info layer to fig
fig.add_layer(markers)

# Show fig (the heatmap from before along with hotel info pins)
fig


Figure(layout=FigureLayout(height='420px'))